In [1]:
# 1 run each of the models and have them output images generated into a separate folder --- unique to the model name

# Faster R-CNN
# FCOS
# RetinaNet
# SSD
# SSDlite

In [98]:
# Importing Utilities
import os
from torchvision.io.image import read_image
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from PIL import ImageFont, ImageDraw
from IPython.display import display
import regex as re
from torchvision.utils import make_grid
import torchvision


# Importing the Models and their respective weights
from torchvision.models.detection import (
    # Faster R-CNN
    fasterrcnn_resnet50_fpn_v2,
    FasterRCNN_ResNet50_FPN_V2_Weights,
    # # FCOS
    fcos_resnet50_fpn,
    FCOS_ResNet50_FPN_Weights,
    # RetinaNet
    retinanet_resnet50_fpn_v2,
    RetinaNet_ResNet50_FPN_V2_Weights,
    # SSD
    ssd300_vgg16,
    SSD300_VGG16_Weights,
    # SSDlite
    ssdlite320_mobilenet_v3_large,
    SSDLite320_MobileNet_V3_Large_Weights,
)

In [117]:
def object_detection(
    model, weights, image_path, output_directory="outputs", threshold=0.9
):
    img = read_image(image_path)
    font_path = os.path.abspath("fonts/OpenSans-Regular.ttf")
    font = ImageFont.truetype(font_path, 30)
    # Step 1: Initialize model with the best available weights
    weights = weights.DEFAULT
    model_name = model.__name__
    model = model(weights=weights, box_score_thresh=threshold)
    model.eval()

    # Step 2: Initialize the inference transforms
    preprocess = weights.transforms()

    # Step 3: Apply inference preprocessing transforms
    batch = [preprocess(img)]

    # Step 4: Use the model and visualize the prediction
    prediction = model(batch)[0]
    labels = [weights.meta["categories"][i] for i in prediction["labels"]]
    scores = prediction["scores"]
    labels_with_scores = [
        f"{label} {score:.2f}" for label, score in zip(labels, scores)
    ]

    box = draw_bounding_boxes(
        img,
        boxes=prediction["boxes"],
        labels=labels_with_scores,
        colors="red",
        width=4,
        font=font_path,
        font_size=20,
    )
    im = to_pil_image(box.detach())

    # Draw the model name at the bottom right of the image
    draw = ImageDraw.Draw(im)
    draw.text((5, 5), model_name, font=font, fill=(57, 255, 20), align="left")

    # creating the general output dirctory
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # creating the model's output directory:
    model_output_directory = os.path.join(output_directory, model_name)
    if not os.path.exists(model_output_directory):
        os.makedirs(model_output_directory)
    # extracting the image base name:
    base_name = os.path.basename(image_path)

    # creating the full image output file name
    output_path = os.path.join(model_output_directory, base_name)
    print(f"Model Output saved to {output_path}")
    im.save(output_path, "JPEG")

In [111]:
# Sort images by their number
def extract_number(filename):
    match = re.findall(r"\d+", filename)
    if match:
        return int(match[0])
    return float("inf")


sorted_filenames = sorted(os.listdir("images"), key=extract_number)
# Printing out the sorted file names
sorted_filenames[:5]

['image_1.jpg', 'image_2.jpg', 'image_3.jpg', 'image_4.jpg', 'image_5.jpg']

In [118]:
for image in sorted_filenames[:5]:
    path = "images/" + image

    object_detection(
        fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights, path
    )
    object_detection(fcos_resnet50_fpn, FCOS_ResNet50_FPN_Weights, path)

    object_detection(retinanet_resnet50_fpn_v2, RetinaNet_ResNet50_FPN_V2_Weights, path)

    object_detection(ssd300_vgg16, SSD300_VGG16_Weights, path)

    object_detection(
        ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights, path
    )

Model Output saved to outputs/fasterrcnn_resnet50_fpn_v2/image_1.jpg
Model Output saved to outputs/fcos_resnet50_fpn/image_1.jpg
Model Output saved to outputs/retinanet_resnet50_fpn_v2/image_1.jpg
Model Output saved to outputs/ssd300_vgg16/image_1.jpg
Model Output saved to outputs/ssdlite320_mobilenet_v3_large/image_1.jpg


/Users/aisaiahpellecer/Desktop/LOC/loc-env/lib/python3.9/site-packages/torchvision/utils.py:209: UserWarning: boxes doesn't contain any box. No box was drawn
  warnings.warn("boxes doesn't contain any box. No box was drawn")


Model Output saved to outputs/fasterrcnn_resnet50_fpn_v2/image_2.jpg
Model Output saved to outputs/fcos_resnet50_fpn/image_2.jpg
Model Output saved to outputs/retinanet_resnet50_fpn_v2/image_2.jpg
Model Output saved to outputs/ssd300_vgg16/image_2.jpg
Model Output saved to outputs/ssdlite320_mobilenet_v3_large/image_2.jpg
Model Output saved to outputs/fasterrcnn_resnet50_fpn_v2/image_3.jpg
Model Output saved to outputs/fcos_resnet50_fpn/image_3.jpg
Model Output saved to outputs/retinanet_resnet50_fpn_v2/image_3.jpg
Model Output saved to outputs/ssd300_vgg16/image_3.jpg
Model Output saved to outputs/ssdlite320_mobilenet_v3_large/image_3.jpg
Model Output saved to outputs/fasterrcnn_resnet50_fpn_v2/image_4.jpg
Model Output saved to outputs/fcos_resnet50_fpn/image_4.jpg
Model Output saved to outputs/retinanet_resnet50_fpn_v2/image_4.jpg
Model Output saved to outputs/ssd300_vgg16/image_4.jpg
Model Output saved to outputs/ssdlite320_mobilenet_v3_large/image_4.jpg
Model Output saved to output

In [119]:
# read images from computer
folders = os.listdir("outputs/")
# image
val = 1
grid_number = len(os.listdir("outputs/fasterrcnn_resnet50_fpn_v2"))
for grid_image in range(grid_number):
    image_vars = []
    for i, folder in enumerate(os.listdir("outputs/")):
        sub_name = "outputs/" + folder
        sub_directory = os.listdir(sub_name)
        # print(sub_directory[0])
        image_vars.append(read_image(sub_name + "/" + sub_directory[grid_image]))

    Grid = make_grid(image_vars)
    img = torchvision.transforms.ToPILImage()(Grid)

    if not os.path.exists("model_panoramas"):
        os.makedirs("model_panoramas")

    panorama_name = "model_panoramas/image" + str(val) + ".jpg"
    img.save(panorama_name, "JPEG")
    val += 1